# Food Image Classifier

This part of the Manning Live project - https://liveproject.manning.com/project/210 . In synposis, By working on this project, I will be classying the food variety of 101 type. Dataset is already availble in public but we will be starting with subset of the classifier

## Dataset

As a general best practice to ALWAYS start with a subset of the dataset rather than a full one. There are two reason for the same
1. As you experiement with the model, You dont want to run over all the dataset that will slow down the process
2. You will end up wasting lots of GPU resources well before the getting best model for the Job

In the Case live Project, The authors already shared the subset of the notebook so we can use the same for the baseline model

In [ ]:
#!wget https://lp-prod-resources.s3-us-west-2.amazonaws.com/other/Deploying+a+Deep+Learning+Model+on+Web+and+Mobile+Applications+Using+TensorFlow/Food+101+-+Data+Subset.zip
#!unzip Food+101+-+Data+Subset.zip

In [1]:
!pip install efficientnet_pytorch -q

In [2]:
import torch
from torchvision import datasets,models
import torchvision.transforms as tt
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data import DataLoader,random_split,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from fastprogress.fastprogress import master_bar, progress_bar
from efficientnet_pytorch import EfficientNet

In [3]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
train_tfms = tt.Compose([tt.RandomHorizontalFlip(),
                         tt.Resize([224,224]),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)])
valid_tfms = tt.Compose([tt.Resize([224,224]),tt.ToTensor(), tt.Normalize(*stats)])

Create a Pytorch dataset from the image folder. This will allow us to create a Training dataset and validation dataset

In [4]:
ds = datasets.ImageFolder('food-101-subset/images/')

In [5]:
class CustomDataset(Dataset):
    def __init__(self,ds,transformer):
        self.ds = ds
        self.transform = transformer
    
    def __getitem__(self,idx):
        image,label = self.ds[idx]
        img = self.transform(image)
        return img,label
    
    def __len__(self):
        return len(ds)

In [6]:
train_len=0.8*len(ds)
val_len = len(ds) - train_len
int(train_len),int(val_len)

(2400, 600)

In [7]:
train_ds,val_ds = random_split(dataset=ds,lengths=[int(train_len),int(val_len)],generator=torch.Generator().manual_seed(42))

In [8]:
t_ds = CustomDataset(train_ds.dataset,train_tfms)
v_ds = CustomDataset(val_ds.dataset,valid_tfms)

In [9]:
batch_size = 32
train_dl = DataLoader(t_ds, batch_size, shuffle=True, pin_memory=True)
valid_dl = DataLoader(v_ds, batch_size, pin_memory=True)

In [10]:
for x,yb in train_dl:
    print(x.shape)
    break;

torch.Size([32, 3, 224, 224])


In [11]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
        break

# Create a ResNet Model with default Parameters

In [50]:
model = EfficientNet.from_pretrained("efficientnet-b0")
head = nn.Sequential(nn.Dropout(0.2),nn.Flatten(),nn.Linear(1280,100),nn.Linear(100,3),MemoryEfficientSwish())

Loaded pretrained weights for efficientnet-b0


In [51]:
los = nn.CrossEntropyLoss()
for x,y in train_dl:
    f = body.extract_features(x)
    f = nn.functional.adaptive_avg_pool2d(f, (1, 1))
    f = torch.flatten(f, 1) 
    yb = head(f)
    loss = los(yb,y)
    print(loss)
    print(y.shape)
    break;

tensor(1.0991, grad_fn=<NllLossBackward>)
torch.Size([32])


In [13]:
from efficientnet_pytorch.utils import MemoryEfficientSwish

In [53]:
class Flatten(nn.Module):
    def forward(self,x):
        return torch.flatten(x,1)

class FoodImageClassifer(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = EfficientNet.from_pretrained("efficientnet-b0")
        self.head = nn.Sequential(nn.Dropout(0.2),nn.Linear(1280,3),MemoryEfficientSwish())
    
    def forward(self,x):
        x = self.model.extract_features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1) 
        return self.head(x)
    
    def freeze(self):
        return true

In [54]:
def fit(epochs,model,train_dl,valid_dl,loss_fn,opt):
    mb = master_bar(range(epochs))
    mb.write(['epoch','train_loss','valid_loss','trn_acc','val_acc'],table=True)

    for i in mb:    
        trn_loss,val_loss = 0.0,0.0
        trn_acc,val_acc = 0,0
        trn_n,val_n = len(train_dl.dataset),len(valid_dl.dataset)
        model.train()
        for xb,yb in progress_bar(train_dl,parent=mb):
            xb,yb = xb.to(device), yb.to(device)
            out = model(xb)
            opt.zero_grad()
            loss = loss_fn(out,yb)
            _,pred = torch.max(out.data, 1)
            trn_acc += (pred == yb).sum().item()
            trn_loss += loss.item()
            loss.backward()
            opt.step()
        trn_loss /= mb.child.total
        trn_acc /= trn_n

        model.eval()
        with torch.no_grad():
            for xb,yb in progress_bar(valid_dl,parent=mb):
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out,yb)
                val_loss += loss.item()
                _,pred = torch.max(out.data, 1)
                val_acc += (pred == yb).sum().item()
        val_loss /= mb.child.total
        val_acc /= val_n

        mb.write([i,f'{trn_loss:.6f}',f'{val_loss:.6f}',f'{trn_acc:.6f}',f'{val_acc:.6f}'],table=True)

# Making the Resnet as a Feature Extractor and training model

In [ ]:
model = FoodImageClassifer()
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#model.freeze()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
fit(10,model=model,train_dl=train_dl,valid_dl=valid_dl,loss_fn=criterion,opt=optimizer_ft)

Loaded pretrained weights for efficientnet-b0


epoch,train_loss,valid_loss,trn_acc,val_acc
0,1.070354,1.024411,0.513667,0.712333
1,0.974153,0.847737,0.749000,0.824333
2,0.749302,0.522657,0.822000,0.859667
3,0.489167,0.337034,0.851000,0.890333
4,0.344713,0.252785,0.882333,0.920000
5,0.279971,0.195080,0.901000,0.937333
6,0.235730,0.159489,0.923000,0.949333
7,0.195900,0.128502,0.933333,0.957667
8,0.171641,0.107370,0.937667,0.965667


# Freeze the layers

In [ ]:
torch.save(model.state_dict,'efficient.pth')